In [1]:
import matplotlib.pyplot as plt
import numpy as np

from src.drawing_utils import Arrow, Point, clear_axes
from src.plotting_utils import configure_matplotlib, rm

configure_matplotlib()

In [ ]:
fig, ax = plt.subplots()
clear_axes(ax)
ax.set_xlim(-1, 12)
ax.set_ylim(-1, 12)
Arrow.horizontal(x2=11).drawn(ax).end.labeled(ax, "$Q$", offset=(3.5, -3))
Arrow.vertical(y2=11).drawn(ax).end.labeled(ax, "$P$")
x = np.linspace(0, 10)
y1 = 1 + 1 * x
y2 = 10 - x
ax.plot(x, y1, label=rm("Supply Curve"))
ax.plot(x, y2, label=rm("Demand Curve"))
Q_star = 4.5
P_star = 5.5
ax.fill_between(
    x,
    y1,
    0,
    where=(x <= Q_star),
    alpha=0.2,
    hatch=r"\\",
    edgecolor="blue",
    label=rm("Seller Cost, $C$"),
)
ax.fill_between(
    x,
    y2,
    0,
    where=(x <= Q_star),
    alpha=0.2,
    hatch="//",
    edgecolor="orange",
    label=rm("Buyer Utility, $U$"),
)
Point(Q_star, P_star).drawn(ax).labeled(
    ax, "$(Q^*, P^*)$", ha="left", offset=(10, -2.5)
)
ax.legend()
fig.savefig("img/fig_2_1.png", dpi=200)